In [9]:
%load_ext autoreload
%autoreload 2

In [16]:
import pandas as pd
import os
from glob import glob
from tqdm import tqdm_notebook as tqdm
pd.set_option('display.max_columns', 200)


In [36]:
#巡回先を作る
#枠とレースIDがあればマージできる？
result_infofile_list = glob("test/*/info_html/*") # 後々フォルダーが変わる
main_info_file = [x.replace("\\", "/") for x in result_infofile_list]

result_resultfile_list = glob("test/*/result_html/*") # 後々フォルダーが変わる
main_result_file = [x.replace("\\", "/") for x in result_resultfile_list]

target_info_list = [] #配列を空にする
target_result_list = [] #配列を空にする

match_up_info_list = [] #配列を空にする
match_up_result_list = [] #配列を空にする

#is_file = os.path.isfile("match_up.txt")

#if is_file:
#    with open("match_up.txt", "r") as tf:
#        match_up_list = tf.read().split(',')[:-1]
#    #途中から始めるためのコード書く
#    target_txts_list = set(result_file) - set(match_up_list)
#else:
#    target_txts_list = result_file

target_info_list = main_info_file[0:10]
target_result_list = main_result_file[0:10]

In [41]:
#データフレームを4個作る
main_info = pd.DataFrame(index=[])
main_info_dummy = pd.DataFrame(index=[])
main_result = pd.DataFrame(index=[])
main_result_dummy = pd.DataFrame(index=[])

dfs_all = pd.DataFrame(index=[])
dfs_dummy = pd.DataFrame(index=[])

In [ ]:
dfs = pd.read_html("row_html/01/info_html/01_20180520_10.html", encoding="utf-8")
display(dfs[0])

In [48]:
for target_info in tqdm(target_info_list):
    dfs = pd.read_html(target_info, encoding="utf-8")
    dfs_dummy = dfs[0]
    dfs_all = dfs_all.append(dfs_dummy)

display(dfs_all.head(1))

C:\Users\rstta\AppData\Local\Temp/ipykernel_28128/3279932081.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for target_info in tqdm(target_info_list):


  0%|          | 0/1000 [00:00<?, ?it/s]

Unnamed: 8_level_0           ボート       ボートレーサー      \
  Unnamed: 8_level_1      No2連率3連率      F数L数平均ST  写真   
  Unnamed: 8_level_2      No2連率3連率      F数L数平均ST  写真   
0                NaN  49  33.90  -  F0  L0  0.22 NaN   

                                             モーター レースNo（艇番色）進入コースSTタイミング成績  \
                登録番号/級別氏名支部/出身地年齢/体重     No2連率3連率 レースNo（艇番色）進入コースSTタイミング成績   
                登録番号/級別氏名支部/出身地年齢/体重     No2連率3連率      Unnamed: 19_level_2   
0  3115  / B1 鈴木 正人 群馬/－  52歳/50.0kg  70  0.00  -                      NaN   

                                                                              \
                                                                               
  Unnamed: 20_level_2 Unnamed: 21_level_2 Unnamed: 22_level_2   初日 初日.1  最終日   
0                 NaN                 NaN                 NaN  NaN  NaN  NaN   

                                                                     全国  \
                                                               勝率2連率3連率   
  最終日.1  ２日目 ２日目.1  ３日目 ３日目.1  ４日目 ４日目.1  ５日目 ５日目.1            勝率2連率3連率   
0   NaN  NaN   NaN  NaN   NaN  NaN   NaN  NaN   NaN  3.94  16.10  33.80   

                   当地   早見  枠  
             勝率2連率3連率   早見  枠  
             勝率2連率3連率   早見  枠  
0  3.96  15.40  35.20  NaN  １

In [ ]:
display(dfs_all.head(1))

In [74]:
#dfs_all[("レースNo（艇番色）進入コースSTタイミング成績", "レースNo（艇番色）進入コースSTタイミング成績", "Unnamed: 19_level_2")].isna().sum()
print(len(dfs_all))

48000


In [57]:
display(dfs_all["ボートレーサー"])

,F数L数平均ST,写真,登録番号/級別氏名支部/出身地年齢/体重
,F数L数平均ST,写真,登録番号/級別氏名支部/出身地年齢/体重
0,F0 L0 0.22,NaN,3115 / B1 鈴木 正人 群馬/－ 52歳/50.0kg
1,F0 L0 0.22,NaN,3115 / B1 鈴木 正人 群馬/－ 52歳/50.0kg
2,F0 L0 0.22,NaN,3115 / B1 鈴木 正人 群馬/－ 52歳/50.0kg
3,F0 L0 0.22,NaN,3115 / B1 鈴木 正人 群馬/－ 52歳/50.0kg
4,F0 L0 0.19,NaN,3560 / B1 太田 克哉 群馬/群馬 45歳/49.0kg
...,...,...,...
19,F0 L0 0.16,NaN,4478 / A2 櫻本 あゆみ 群馬/栃木 26歳/49.6kg
20,F1 L0 0.20,NaN,4699 / B2 八十岡 恵美 福井/埼玉 27歳/47.0kg
21,F1 L0 0.20,NaN,4699 / B2 八十岡 恵美 福井/埼玉 27歳/47.0kg


In [34]:
# info_htmlの加工をする
# race_id、枠、選手ID、級別、F数、L数、平均ST、モーター2連、モーター3連、初日進入、初日ST、初日成績、～ 、最終日、
# 体重、調整、展示タイム、チルト、プロペラ、部品交換、前走成績、

for target_info in tqdm(target_info_list):
    #レース結果を保存したhtmlファイルとそのDF
    dfs = pd.read_html(target_info, encoding="utf-8")
    x = dfs[0]
    display(x)
    #x_ = x.iloc[i:i+2,0:2]
    #display[x_]
#dfs = pd.read_html(target_info_list[0])
#for i in range(0, ):

display(x)
#display(x.head())
#display(x.iloc[1,2])

C:\Users\rstta\AppData\Local\Temp/ipykernel_28128/1537428817.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for target_info in tqdm(target_info_list):


  0%|          | 0/4 [00:00<?, ?it/s]

枠 ボートレーサー                                                    \
    枠      写真                登録番号/級別氏名支部/出身地年齢/体重      F数L数平均ST   
    枠      写真                登録番号/級別氏名支部/出身地年齢/体重      F数L数平均ST   
0   １     NaN   3115  / B1 鈴木 正人 群馬/－  52歳/50.0kg  F0  L0  0.22   
4   ２     NaN  3560  / B1 太田 克哉 群馬/群馬  45歳/49.0kg  F0  L0  0.19   
8   ３     NaN  3376  / B1 萩原 善行 群馬/群馬  46歳/48.5kg  F0  L0  0.24   
12  ４     NaN   3532  / A1 柴田 光 群馬/群馬  42歳/52.8kg  F0  L0  0.17   
16  ５     NaN    2808  / B2 田嶋 登 群馬/－  58歳/51.4kg  F0  L0  0.29   
20  ６     NaN  4645  / B1 上村 純一 群馬/千葉  33歳/50.4kg  F0  L0  0.16   

                    全国                  当地          モーター           ボート  \
              勝率2連率3連率            勝率2連率3連率      No2連率3連率      No2連率3連率   
              勝率2連率3連率            勝率2連率3連率      No2連率3連率      No2連率3連率   
0   3.94  16.10  33.80  3.96  15.40  35.20   70  0.00  -  49  33.90  -   
4   4.48  22.50  40.00  4.63  24.00  43.00   43  0.00  -  16  39.10  -   
8   4.56  29.10  40.50  4.23  25.70  37.10  35  14.20  -  52  25.00  -   
12  6.23  46.00  61.70  6.80  54.10  69.70   60  0.00  -  71  52.80  -   
16    2.32  2.70  8.10    2.27  0.00  7.60   29  0.00  -  33  31.30  -   
20  5.17  33.30  53.30  4.40  22.50  39.70  27  66.60  -  35  24.40  -   

   Unnamed: 8_level_0 レースNo（艇番色）進入コースSTタイミング成績                               \
   Unnamed: 8_level_1 レースNo（艇番色）進入コースSTタイミング成績                                
   Unnamed: 8_level_2                       初日 初日.1 ２日目 ２日目.1 ３日目 ３日目.1 ４日目   
0                 NaN                      NaN  NaN NaN   NaN NaN   NaN NaN   
4                 NaN                      NaN  NaN NaN   NaN NaN   NaN NaN   
8                 NaN                      NaN  NaN NaN   NaN NaN   NaN NaN   
12                NaN                      NaN  NaN NaN   NaN NaN   NaN NaN   
16                NaN                      NaN  NaN NaN   NaN NaN   NaN NaN   
20                NaN                      NaN  NaN NaN   NaN NaN   NaN NaN   

                                                                       早見  
                                                                       早見  
   ４日目.1 ５日目 ５日目.1 最終日 最終日.1 Unnamed: 21_level_2 Unnamed: 22_level_2   早見  
0    NaN NaN   NaN NaN   NaN                 NaN                 NaN  NaN  
4    NaN NaN   NaN NaN   NaN                 NaN                 NaN   8R  
8    NaN NaN   NaN NaN   NaN                 NaN                 NaN  NaN  
12   NaN NaN   NaN NaN   NaN                 NaN                 NaN  11R  
16   NaN NaN   NaN NaN   NaN                 NaN                 NaN  NaN  
20   NaN NaN   NaN NaN   NaN                 NaN                 NaN   7R

枠 ボートレーサー                                                    \
    枠      写真                登録番号/級別氏名支部/出身地年齢/体重      F数L数平均ST   
    枠      写真                登録番号/級別氏名支部/出身地年齢/体重      F数L数平均ST   
0   １     NaN  3310  / B1 金澤 一洋 群馬/群馬  49歳/50.0kg  F0  L0  0.18   
4   ２     NaN  2528  / B1 新井 敏司 群馬/栃木  65歳/50.4kg  F0  L0  0.17   
8   ３     NaN   2841  / A2 吉田 稔 群馬/群馬  57歳/52.9kg  F0  L0  0.18   
12  ４     NaN   3973  / B1 崎 利仁 群馬/栃木  37歳/53.4kg  F0  L0  0.19   
16  ５     NaN   3496  / B1 赤坂 等 群馬/群馬  45歳/51.3kg  F0  L0  0.18   
20  ６     NaN  4539  / A2 金子 賢志 群馬/群馬  23歳/49.0kg  F0  L0  0.14   

                    全国                  当地          モーター           ボート  \
              勝率2連率3連率            勝率2連率3連率      No2連率3連率      No2連率3連率   
              勝率2連率3連率            勝率2連率3連率      No2連率3連率      No2連率3連率   
0   4.40  21.70  43.40  4.56  22.70  39.30   65  0.00  -  45  40.80  -   
4   5.71  37.90  63.20  5.14  29.60  50.00  53  33.30  -  63  40.00  -   
8   5.65  32.00  61.10  6.23  42.30  65.30  37  22.20  -  64  50.90  -   
12  5.19  35.50  50.00  5.72  33.80  61.50  57  66.60  -  21  34.60  -   
16  4.61  26.30  41.60  4.57  22.60  41.50   58  0.00  -  30  29.00  -   
20  6.35  48.80  65.10  5.82  46.30  56.70   62  0.00  -  11  44.60  -   

   Unnamed: 8_level_0 レースNo（艇番色）進入コースSTタイミング成績                               \
   Unnamed: 8_level_1 レースNo（艇番色）進入コースSTタイミング成績                                
   Unnamed: 8_level_2                       初日 初日.1 ２日目 ２日目.1 ３日目 ３日目.1 ４日目   
0                 NaN                      NaN  NaN NaN   NaN NaN   NaN NaN   
4                 NaN                      NaN  NaN NaN   NaN NaN   NaN NaN   
8                 NaN                      NaN  NaN NaN   NaN NaN   NaN NaN   
12                NaN                      NaN  NaN NaN   NaN NaN   NaN NaN   
16                NaN                      NaN  NaN NaN   NaN NaN   NaN NaN   
20                NaN                      NaN  NaN NaN   NaN NaN   NaN NaN   

                                                                       早見  
                                                                       早見  
   ４日目.1 ５日目 ５日目.1 最終日 最終日.1 Unnamed: 21_level_2 Unnamed: 22_level_2   早見  
0    NaN NaN   NaN NaN   NaN                 NaN                 NaN  NaN  
4    NaN NaN   NaN NaN   NaN                 NaN                 NaN  10R  
8    NaN NaN   NaN NaN   NaN                 NaN                 NaN   9R  
12   NaN NaN   NaN NaN   NaN                 NaN                 NaN   7R  
16   NaN NaN   NaN NaN   NaN                 NaN                 NaN  NaN  
20   NaN NaN   NaN NaN   NaN                 NaN                 NaN  11R

枠 ボートレーサー                                                     \
    枠      写真                 登録番号/級別氏名支部/出身地年齢/体重      F数L数平均ST   
    枠      写真                 登録番号/級別氏名支部/出身地年齢/体重      F数L数平均ST   
0   １     NaN   3482  / B1 金子 猛志 群馬/群馬  42歳/57.4kg  F0  L0  0.17   
4   ２     NaN   3850  / B1 小川 知行 群馬/群馬  41歳/53.1kg  F0  L0  0.18   
8   ３     NaN  3947  / A2 寺本 昇平 群馬/神奈川  37歳/48.5kg  F0  L0  0.17   
12  ４     NaN    4491  / B1 田中 堅 群馬/群馬  26歳/51.1kg  F0  L0  0.20   
16  ５     NaN  4420  / B1 津久井　拓也 群馬/栃木  26歳/51.4kg  F0  L0  0.17   
20  ６     NaN  3686  / A2 鳥居塚　孝博 群馬/群馬  38歳/51.3kg  F0  L0  0.16   

                    全国                  当地          モーター           ボート  \
              勝率2連率3連率            勝率2連率3連率      No2連率3連率      No2連率3連率   
              勝率2連率3連率            勝率2連率3連率      No2連率3連率      No2連率3連率   
0   3.81  20.20  33.70  4.96  32.00  49.00   68  0.00  -  43  38.70  -   
4   4.29  23.70  39.60  5.37  27.30  55.90   16  0.00  -  36  40.70  -   
8   5.71  35.70  60.70  6.37  51.00  63.80   67  0.00  -  53  39.50  -   
12  4.33  20.50  42.40  4.30  20.20  42.80  21  25.00  -  47  28.00  -   
16  4.63  23.30  42.20  5.53  31.80  62.10   24  0.00  -  28  44.20  -   
20  5.57  32.00  58.00  6.42  51.50  65.10  40  75.00  -  59  32.70  -   

   Unnamed: 8_level_0 レースNo（艇番色）進入コースSTタイミング成績                               \
   Unnamed: 8_level_1 レースNo（艇番色）進入コースSTタイミング成績                                
   Unnamed: 8_level_2                       初日 初日.1 ２日目 ２日目.1 ３日目 ３日目.1 ４日目   
0                 NaN                      NaN  NaN NaN   NaN NaN   NaN NaN   
4                 NaN                      NaN  NaN NaN   NaN NaN   NaN NaN   
8                 NaN                      NaN  NaN NaN   NaN NaN   NaN NaN   
12                NaN                      NaN  NaN NaN   NaN NaN   NaN NaN   
16                NaN                      NaN  NaN NaN   NaN NaN   NaN NaN   
20                NaN                      NaN  NaN NaN   NaN NaN   NaN NaN   

                                                                       早見  
                                                                       早見  
   ４日目.1 ５日目 ５日目.1 最終日 最終日.1 Unnamed: 21_level_2 Unnamed: 22_level_2   早見  
0    NaN NaN   NaN NaN   NaN                 NaN                 NaN  NaN  
4    NaN NaN   NaN NaN   NaN                 NaN                 NaN   7R  
8    NaN NaN   NaN NaN   NaN                 NaN                 NaN  11R  
12   NaN NaN   NaN NaN   NaN                 NaN                 NaN   8R  
16   NaN NaN   NaN NaN   NaN                 NaN                 NaN   9R  
20   NaN NaN   NaN NaN   NaN                 NaN                 NaN  10R

枠 ボートレーサー                                                    \
    枠      写真                登録番号/級別氏名支部/出身地年齢/体重      F数L数平均ST   
    枠      写真                登録番号/級別氏名支部/出身地年齢/体重      F数L数平均ST   
0   １     NaN   3438  / B1 須田 秀一 群馬/－  44歳/50.7kg  F0  L0  0.22   
4   ２     NaN  4305  / A2 金子 拓矢 群馬/栃木  30歳/50.6kg  F0  L0  0.14   
8   ３     NaN  4407  / B1 鹿島 敏弘 群馬/青森  27歳/52.4kg  F0  L0  0.22   
12  ４     NaN  4233  / A2 亀山 雅幸 群馬/群馬  30歳/50.0kg  F0  L0  0.16   
16  ５     NaN  4203  / B1 小倉 康典 群馬/群馬  31歳/52.0kg  F0  L0  0.17   
20  ６     NaN   4486  / B2 野村 誠 群馬/群馬  27歳/50.3kg  F0  L0  0.23   

                    全国                  当地          モーター           ボート  \
              勝率2連率3連率            勝率2連率3連率      No2連率3連率      No2連率3連率   
              勝率2連率3連率            勝率2連率3連率      No2連率3連率      No2連率3連率   
0   4.81  30.10  46.20  4.94  29.60  50.00  18  14.20  -  32  34.10  -   
4   5.98  34.80  61.60  6.08  43.60  61.90   25  0.00  -  54  38.00  -   
8   4.44  28.00  41.40  5.13  34.60  50.60  28  37.50  -  56  36.30  -   
12  5.71  37.10  53.60  6.34  52.00  64.30   61  0.00  -  27  30.20  -   
16  4.93  34.00  44.60  4.77  28.50  42.80  36  14.20  -  38  42.50  -   
20  4.78  26.00  50.70  4.38  25.60  48.60  34  22.20  -  13  28.00  -   

   Unnamed: 8_level_0 レースNo（艇番色）進入コースSTタイミング成績                               \
   Unnamed: 8_level_1 レースNo（艇番色）進入コースSTタイミング成績                                
   Unnamed: 8_level_2                       初日 初日.1 ２日目 ２日目.1 ３日目 ３日目.1 ４日目   
0                 NaN                      NaN  NaN NaN   NaN NaN   NaN NaN   
4                 NaN                      NaN  NaN NaN   NaN NaN   NaN NaN   
8                 NaN                      NaN  NaN NaN   NaN NaN   NaN NaN   
12                NaN                      NaN  NaN NaN   NaN NaN   NaN NaN   
16                NaN                      NaN  NaN NaN   NaN NaN   NaN NaN   
20                NaN                      NaN  NaN NaN   NaN NaN   NaN NaN   

                                                                       早見  
                                                                       早見  
   ４日目.1 ５日目 ５日目.1 最終日 最終日.1 Unnamed: 21_level_2 Unnamed: 22_level_2   早見  
0    NaN NaN   NaN NaN   NaN                 NaN                 NaN  NaN  
4    NaN NaN   NaN NaN   NaN                 NaN                 NaN   9R  
8    NaN NaN   NaN NaN   NaN                 NaN                 NaN  NaN  
12   NaN NaN   NaN NaN   NaN                 NaN                 NaN  10R  
16   NaN NaN   NaN NaN   NaN                 NaN                 NaN   8R  
20   NaN NaN   NaN NaN   NaN                 NaN                 NaN  NaN

枠 ボートレーサー                                                    \
    枠      写真                登録番号/級別氏名支部/出身地年齢/体重      F数L数平均ST   
    枠      写真                登録番号/級別氏名支部/出身地年齢/体重      F数L数平均ST   
0   １     NaN   3438  / B1 須田 秀一 群馬/－  44歳/50.7kg  F0  L0  0.22   
1   １     NaN   3438  / B1 須田 秀一 群馬/－  44歳/50.7kg  F0  L0  0.22   
2   １     NaN   3438  / B1 須田 秀一 群馬/－  44歳/50.7kg  F0  L0  0.22   
3   １     NaN   3438  / B1 須田 秀一 群馬/－  44歳/50.7kg  F0  L0  0.22   
4   ２     NaN  4305  / A2 金子 拓矢 群馬/栃木  30歳/50.6kg  F0  L0  0.14   
5   ２     NaN  4305  / A2 金子 拓矢 群馬/栃木  30歳/50.6kg  F0  L0  0.14   
6   ２     NaN  4305  / A2 金子 拓矢 群馬/栃木  30歳/50.6kg  F0  L0  0.14   
7   ２     NaN  4305  / A2 金子 拓矢 群馬/栃木  30歳/50.6kg  F0  L0  0.14   
8   ３     NaN  4407  / B1 鹿島 敏弘 群馬/青森  27歳/52.4kg  F0  L0  0.22   
9   ３     NaN  4407  / B1 鹿島 敏弘 群馬/青森  27歳/52.4kg  F0  L0  0.22   
10  ３     NaN  4407  / B1 鹿島 敏弘 群馬/青森  27歳/52.4kg  F0  L0  0.22   
11  ３     NaN  4407  / B1 鹿島 敏弘 群馬/青森  27歳/52.4kg  F0  L0  0.22   
12  ４     NaN  4233  / A2 亀山 雅幸 群馬/群馬  30歳/50.0kg  F0  L0  0.16   
13  ４     NaN  4233  / A2 亀山 雅幸 群馬/群馬  30歳/50.0kg  F0  L0  0.16   
14  ４     NaN  4233  / A2 亀山 雅幸 群馬/群馬  30歳/50.0kg  F0  L0  0.16   
15  ４     NaN  4233  / A2 亀山 雅幸 群馬/群馬  30歳/50.0kg  F0  L0  0.16   
16  ５     NaN  4203  / B1 小倉 康典 群馬/群馬  31歳/52.0kg  F0  L0  0.17   
17  ５     NaN  4203  / B1 小倉 康典 群馬/群馬  31歳/52.0kg  F0  L0  0.17   
18  ５     NaN  4203  / B1 小倉 康典 群馬/群馬  31歳/52.0kg  F0  L0  0.17   
19  ５     NaN  4203  / B1 小倉 康典 群馬/群馬  31歳/52.0kg  F0  L0  0.17   
20  ６     NaN   4486  / B2 野村 誠 群馬/群馬  27歳/50.3kg  F0  L0  0.23   
21  ６     NaN   4486  / B2 野村 誠 群馬/群馬  27歳/50.3kg  F0  L0  0.23   
22  ６     NaN   4486  / B2 野村 誠 群馬/群馬  27歳/50.3kg  F0  L0  0.23   
23  ６     NaN   4486  / B2 野村 誠 群馬/群馬  27歳/50.3kg  F0  L0  0.23   

                    全国                  当地          モーター           ボート  \
              勝率2連率3連率            勝率2連率3連率      No2連率3連率      No2連率3連率   
              勝率2連率3連率            勝率2連率3連率      No2連率3連率      No2連率3連率   
0   4.81  30.10  46.20  4.94  29.60  50.00  18  14.20  -  32  34.10  -   
1   4.81  30.10  46.20  4.94  29.60  50.00  18  14.20  -  32  34.10  -   
2   4.81  30.10  46.20  4.94  29.60  50.00  18  14.20  -  32  34.10  -   
3   4.81  30.10  46.20  4.94  29.60  50.00  18  14.20  -  32  34.10  -   
4   5.98  34.80  61.60  6.08  43.60  61.90   25  0.00  -  54  38.00  -   
5   5.98  34.80  61.60  6.08  43.60  61.90   25  0.00  -  54  38.00  -   
6   5.98  34.80  61.60  6.08  43.60  61.90   25  0.00  -  54  38.00  -   
7   5.98  34.80  61.60  6.08  43.60  61.90   25  0.00  -  54  38.00  -   
8   4.44  28.00  41.40  5.13  34.60  50.60  28  37.50  -  56  36.30  -   
9   4.44  28.00  41.40  5.13  34.60  50.60  28  37.50  -  56  36.30  -   
10  4.44  28.00  41.40  5.13  34.60  50.60  28  37.50  -  56  36.30  -   
11  4.44  28.00  41.40  5.13  34.60  50.60  28  37.50  -  56  36.30  -   
12  5.71  37.10  53.60  6.34  52.00  64.30   61  0.00  -  27  30.20  -   
13  5.71  37.10  53.60  6.34  52.00  64.30   61  0.00  -  27  30.20  -   
14  5.71  37.10  53.60  6.34  52.00  64.30   61  0.00  -  27  30.20  -   
15  5.71  37.10  53.60  6.34  52.00  64.30   61  0.00  -  27  30.20  -   
16  4.93  34.00  44.60  4.77  28.50  42.80  36  14.20  -  38  42.50  -   
17  4.93  34.00  44.60  4.77  28.50  42.80  36  14.20  -  38  42.50  -   
18  4.93  34.00  44.60  4.77  28.50  42.80  36  14.20  -  38  42.50  -   
19  4.93  34.00  44.60  4.77  28.50  42.80  36  14.20  -  38  42.50  -   
20  4.78  26.00  50.70  4.38  25.60  48.60  34  22.20  -  13  28.00  -   
21  4.78  26.00  50.70  4.38  25.60  48.60  34  22.20  -  13  28.00  -   
22  4.78  26.00  50.70  4.38  25.60  48.60  34  22.20  -  13  28.00  -   
23  4.78  26.00  50.70  4.38  25.60  48.60  34  22.20  -  13  28.00  -   

   Unnamed: 8_level_0 レースNo（艇番色）進入コースSTタイミング成績                               \
   Unnamed: 8_level_1 レースNo（艇番色）進入コースSTタイミング成績                                
   Unnamed: 8_level_2                

In [ ]:
# result_htmlの加工をする